In [ ]:
# Statistical Physics Machine Learning Project: Phase Transition Prediction

## Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

## Load dataset (placeholder for Monte Carlo simulation data)
# For demonstration, we generate synthetic data simulating spin configurations.
# Replace this with actual dataset loading (e.g., pd.read_csv).
def generate_synthetic_data(n_samples=10000, grid_size=10):
    """Generate synthetic Ising model data."""
    X = np.random.choice([-1, 1], size=(n_samples, grid_size, grid_size))
    temperatures = np.random.uniform(0.5, 5.0, size=n_samples)
    critical_temp = 2.5
    labels = (temperatures > critical_temp).astype(int)  # Binary classification: below/above critical temperature
    return X, temperatures, labels

X, temperatures, y = generate_synthetic_data()

# Reshape data for model
X = X[..., np.newaxis]  # Add channel dimension

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize temperatures (optional, depending on dataset)
scaler = StandardScaler()
temperatures_train = scaler.fit_transform(temperatures[:X_train.shape[0]].reshape(-1, 1))
temperatures_test = scaler.transform(temperatures[X_train.shape[0]:].reshape(-1, 1))

## Define model architecture
def create_model(dropout_rate=0.5, learning_rate=0.001):
    """Build a Convolutional Neural Network."""
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(10, 10, 1)),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

## Hyperparameter Optimization
# Define search space
space = [
    Real(0.1, 0.6, name='dropout_rate'),
    Real(1e-4, 1e-2, name='learning_rate', prior='log-uniform')
]

def objective(params):
    """Objective function for Bayesian optimization."""
    dropout_rate, learning_rate = params
    model = create_model(dropout_rate, learning_rate)
    model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=0)  # Use fewer epochs for speed in optimization
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return -accuracy

# Perform optimization
results = gp_minimize(objective, space, n_calls=10, random_state=42)

# Extract best hyperparameters
best_dropout_rate, best_learning_rate = results.x
print(f"Optimal parameters: Dropout Rate={best_dropout_rate}, Learning Rate={best_learning_rate}")

## Train final model
final_model = create_model(best_dropout_rate, best_learning_rate)
history = final_model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

## Evaluate and visualize results
loss, accuracy = final_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Plot accuracy curve
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

# Confusion matrix
predictions = (final_model.predict(X_test) > 0.5).astype(int)
conf_matrix = confusion_matrix(y_test, predictions)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Classification report
print(classification_report(y_test, predictions))
